   # Lab 2 - Template

The template is given as starting point to help you get up to speed with loading file and understanding data.  
Use the template and answer each question in the appropriate cell.

Submit:
- eLearning
- Github: create new folder Lab-2. Commit your work under the folder

Naming: First-name_Last_name_ID_Lab_2.ipynb

## Dataset description

The dataset for Lab-2 is sample of the SBA dataset posted on Kaggle. 

The dataset is from the U.S. Small Business Administration (SBA)
The U.S. SBA was founded in 1953 on the principle of promoting and assisting small enterprises in the U.S. credit market 
(SBA Overview and History, US Small Business Administration (2015)). Small businesses have been a primary source of job creation in the United States; therefore, fostering small business formation and growth has social benefits by creating job opportunities and reducing unemployment.
There have been many success stories of start-ups receiving SBA loan guarantees such as FedEx and Apple Computer. 
However, there have also been stories of small businesses and/or start-ups that have defaulted on their SBA-guaranteed loans.

More info on the original dataset: https://www.kaggle.com/mirbektoktogaraev/should-this-loan-be-approved-or-denied


In [1]:
import pandas as pd
pd.set_option('display.max_columns', 1500)

import warnings
warnings.filterwarnings('ignore')

#Extend cell width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

## The function `describe_more` is data exploration utility

It is up to you if you want to use the function. If you find bugs, please let me know but try to fix them yourself.  
The function most definetely doesn't work with every input

In [2]:
"""
Created on Mon Mar 18 18:25:50 2019

@author: Uri Smashnov

Purpose: Analyze input Pandas DataFrame and return stats per column
Details: The function calculates levels for categorical variables and allows to analyze summarized information

To view wide table set following Pandas options:
pd.set_option('display.width', 1000)
pd.set_option('max_colwidth',200)
"""
import pandas as pd
def describe_more(df,normalize_ind=False, weight_column=None, skip_columns=[], dropna=True):
    var = [] ; l = [] ; t = []; unq =[]; min_l = []; max_l = [];
    assert isinstance(skip_columns, list), "Argument skip_columns should be list"
    if weight_column is not None:
        if weight_column not in list(df.columns):
            raise AssertionError('weight_column is not a valid column name in the input DataFrame')
      
    for x in df:
        if x in skip_columns:
            pass
        else:
            var.append( x )
            uniq_counts = len(pd.value_counts(df[x],dropna=dropna))
            uniq_counts = len(pd.value_counts(df[x], dropna=dropna)[pd.value_counts(df[x],dropna=dropna)>0])
            l.append(uniq_counts)
            t.append( df[ x ].dtypes )
            min_l.append(df[x].apply(str).str.len().min())
            max_l.append(df[x].apply(str).str.len().max())
            if weight_column is not None and x not in skip_columns:
                df2 = df.groupby(x).agg({weight_column: 'sum'}).sort_values(weight_column, ascending=False)
                df2['authtrans_vts_cnt']=((df2[weight_column])/df2[weight_column].sum()).round(2)
                unq.append(df2.head(n=100).to_dict()[weight_column])
            else:
                df_cat_d = df[x].value_counts(normalize=normalize_ind,dropna=dropna).round(decimals=2)
                df_cat_d = df_cat_d[df_cat_d>0]
                #unq.append(df[x].value_counts().iloc[0:100].to_dict())
                unq.append(df_cat_d.iloc[0:200].to_dict())
                
            
    levels = pd.DataFrame( { 'A_Variable' : var , 'Levels' : l , 'Datatype' : t ,
                             'Min Length' : min_l,
                             'Max Length': max_l,
                             'Level_Values' : unq} )
    #levels.sort_values( by = 'Levels' , inplace = True )
    return levels

## Load data

We assume data is located in the folder `data` where we run our notebook

In [3]:
data = pd.read_csv('SBA_loans_small.csv')
data.head()

,City,State,Zip,Bank,BankState,NAICS,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementGross,BalanceGross,MIS_Status,GrAppv,SBA_Appv
0,BOULDER,CO,80302,WELLS FARGO BANK NATL ASSOC,SD,327122,39,8,1.0,0,8,1,1,Y,N,"$60,639.00",$0.00,CHGOFF,"$30,000.00","$15,000.00"
1,SEATTLE,WA,98101,HANMI BANK,CA,722211,15,2,1.0,0,2,1,1,0,N,"$66,000.00",$0.00,CHGOFF,"$66,000.00","$33,000.00"
2,PHOENIX,AZ,85023,CAPITAL ONE NATL ASSOC,VA,323119,62,15,1.0,0,0,1,1,N,N,"$50,000.00",$0.00,CHGOFF,"$50,000.00","$25,000.00"
3,HOUSTON,TX,77074,JPMORGAN CHASE BANK NATL ASSOC,IL,524210,66,3,1.0,1,3,0,1,Y,N,"$168,853.00",$0.00,CHGOFF,"$50,000.00","$25,000.00"
4,LAS CRUCES,NM,88001,BANK OF AMERICA NATL ASSOC,NM,0,60,1,2.0,0,0,0,0,N,N,"$26,000.00",$0.00,CHGOFF,"$26,000.00","$23,400.00"


In [4]:
data.shape

(448583, 20)

In [5]:
desc_df = describe_more(data)

In [6]:
pd.set_option('display.width', 200)
desc_df.head(n=200)

,A_Variable,Levels,Datatype,Min Length,Max Length,Level_Values
0,City,24757,object,1,30,"{'LOS ANGELES': 5751, 'HOUSTON': 5102, 'NEW YO..."
1,State,51,object,2,3,"{'CA': 65183, 'TX': 35161, 'NY': 28716, 'FL': ..."
2,Zip,28185,int64,1,5,"{90015: 492, 10001: 455, 93401: 385, 90010: 37..."
3,Bank,5156,object,3,30,"{'BANK OF AMERICA NATL ASSOC': 43347, 'WELLS F..."
4,BankState,54,object,2,3,"{'CA': 58768, 'NC': 39709, 'IL': 32982, 'OH': ..."
5,NAICS,1290,int64,1,6,"{0: 100776, 722110: 13888, 722211: 9713, 81111..."
6,Term,392,int64,1,3,"{84: 114519, 60: 44888, 240: 43020, 120: 38908..."
7,NoEmp,466,int64,1,4,"{1: 76992, 2: 68936, 3: 45269, 4: 36503, 5: 30..."
8,NewExist,3,float64,3,3,"{1.0: 321541, 2.0: 126457, 0.0: 520}"
9,CreateJob,196,int64,1,4,"{0: 313743, 1: 31439, 2: 29216, 3: 14422, 4: 1..."


## Question 1

How many of the aech target label are there? Target label is `MIS_Status` column. 

In [7]:
data['MIS_Status'].value_counts()

P I F     369804
CHGOFF     78779
Name: MIS_Status, dtype: int64

## Question 2

Encode target variable `MIS_Status` into `0` and `1`, when value `P I F` is mapped to `0` and `CHGOFF` is mapped to `1`.   

Replace values in the original column

In [8]:
data["MIS_Status"].replace({"P I F" : 0, "CHGOFF": 1}, inplace=True)

## Question 3

Think about how to encode `ZIP` column? 

- Should we leave it as is? 
- Should we encode it using one-hot-encoding?
- Should we encode it using target encoder?

#### Question 3a

Copy original Pandas dataframe into new dataframe.  
Encode `ZIP` column using `from sklearn.preprocessing import OneHotEncoder`  
https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html

Check new dataframe. Is it good idea to use one-hot-encoder on `ZIP` column? 

In [9]:
from sklearn.preprocessing import OneHotEncoder
data2 = data 
ohe = OneHotEncoder(sparse=False)
val=ohe.fit_transform(data2["Zip"].values.reshape(-1, 1))
val.size

12643311855

### Question 3b

Encode ZIP column using target (frequency) encoding technique. 
Although, not entirely  correct, replace `ZIP` values with ratios of 0/1 in the `MIS_Status` column.
For example, if for ZIP=11222 there 100 records with label `0` and 50 records with label `1` replace with value 50/100=0.5 

Please see video on feature engineering topic posted in Module-2 for the explanation why frequency encoding is not ideal without modifications.

In [10]:
data["Zip_encoded"] = data.groupby("Zip")["MIS_Status"].transform("mean")

data[["Zip", "MIS_Status", "Zip_encoded"]].head(10)

,Zip,MIS_Status,Zip_encoded
0,80302,1,0.113636
1,98101,1,0.080882
2,85023,1,0.153846
3,77074,1,0.247191
4,88001,1,0.049645
5,44708,1,0.204545
6,83404,1,0.119403
7,48180,1,0.281250
8,78502,1,0.294118
9,85238,1,1.000000


## Question 4

Encode `NewExist` column using `from sklearn import preprocessing.LabelEncoder`. Replace original column. 

In [11]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
data['NewExist']=le.fit_transform(data['NewExist'])
data

,City,State,Zip,Bank,BankState,NAICS,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementGross,BalanceGross,MIS_Status,GrAppv,SBA_Appv,Zip_encoded
0,BOULDER,CO,80302,WELLS FARGO BANK NATL ASSOC,SD,327122,39,8,1,0,8,1,1,Y,N,"$60,639.00",$0.00,1,"$30,000.00","$15,000.00",0.113636
1,SEATTLE,WA,98101,HANMI BANK,CA,722211,15,2,1,0,2,1,1,0,N,"$66,000.00",$0.00,1,"$66,000.00","$33,000.00",0.080882
2,PHOENIX,AZ,85023,CAPITAL ONE NATL ASSOC,VA,323119,62,15,1,0,0,1,1,N,N,"$50,000.00",$0.00,1,"$50,000.00","$25,000.00",0.153846
3,HOUSTON,TX,77074,JPMORGAN CHASE BANK NATL ASSOC,IL,524210,66,3,1,1,3,0,1,Y,N,"$168,853.00",$0.00,1,"$50,000.00","$25,000.00",0.247191
4,LAS CRUCES,NM,88001,BANK OF AMERICA NATL ASSOC,NM,0,60,1,2,0,0,0,0,N,N,"$26,000.00",$0.00,1,"$26,000.00","$23,400.00",0.049645
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
448578,HUNTINGTON BEACH,CA,92649,WELLS FARGO BANK NATL ASSOC,SD,0,300,10,1,0,0,1,0,N,N,"$492,000.00",$0.00,0,"$492,000.00","$344,400.00",0.125000
448579,DOVER,NH,3820,CITIZENS BANK NATL ASSOC,RI,812112,36,1,2,0,1,0,1,N,N,"$10,000.00",$0.00,0,"$10,000.00","$5,000.00",0.081967
448580,LUBBOCK,TX,79423,PLAINSCAPITAL BANK,TX,611691,64,1,2,1,1,52007,1,N,N,"$68,000.00",$0.00,0,"$68,000.00","$57,800.00",0.080808
448581,SULLIVAN,IL,61951,FIRST MID-ILLINOIS BK & TRUST,IL,0,60,2,1,0,0,1,0,N,Y,"$23,200.00",$0.00,0,"$23,200.00","$18,560.00",0.111111


## Question 5

Fill all columns with missing values with `Unknown` or equivalent value.

```
# filling with Unknown class
df_clean = df.fillna("Unknown")
df_clean
```

In [12]:
for col in data.columns:
    if data[col].isna().any() == True:
        data[col].fillna("Unknown", inplace = True)
data

,City,State,Zip,Bank,BankState,NAICS,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementGross,BalanceGross,MIS_Status,GrAppv,SBA_Appv,Zip_encoded
0,BOULDER,CO,80302,WELLS FARGO BANK NATL ASSOC,SD,327122,39,8,1,0,8,1,1,Y,N,"$60,639.00",$0.00,1,"$30,000.00","$15,000.00",0.113636
1,SEATTLE,WA,98101,HANMI BANK,CA,722211,15,2,1,0,2,1,1,0,N,"$66,000.00",$0.00,1,"$66,000.00","$33,000.00",0.080882
2,PHOENIX,AZ,85023,CAPITAL ONE NATL ASSOC,VA,323119,62,15,1,0,0,1,1,N,N,"$50,000.00",$0.00,1,"$50,000.00","$25,000.00",0.153846
3,HOUSTON,TX,77074,JPMORGAN CHASE BANK NATL ASSOC,IL,524210,66,3,1,1,3,0,1,Y,N,"$168,853.00",$0.00,1,"$50,000.00","$25,000.00",0.247191
4,LAS CRUCES,NM,88001,BANK OF AMERICA NATL ASSOC,NM,0,60,1,2,0,0,0,0,N,N,"$26,000.00",$0.00,1,"$26,000.00","$23,400.00",0.049645
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
448578,HUNTINGTON BEACH,CA,92649,WELLS FARGO BANK NATL ASSOC,SD,0,300,10,1,0,0,1,0,N,N,"$492,000.00",$0.00,0,"$492,000.00","$344,400.00",0.125000
448579,DOVER,NH,3820,CITIZENS BANK NATL ASSOC,RI,812112,36,1,2,0,1,0,1,N,N,"$10,000.00",$0.00,0,"$10,000.00","$5,000.00",0.081967
448580,LUBBOCK,TX,79423,PLAINSCAPITAL BANK,TX,611691,64,1,2,1,1,52007,1,N,N,"$68,000.00",$0.00,0,"$68,000.00","$57,800.00",0.080808
448581,SULLIVAN,IL,61951,FIRST MID-ILLINOIS BK & TRUST,IL,0,60,2,1,0,0,1,0,N,Y,"$23,200.00",$0.00,0,"$23,200.00","$18,560.00",0.111111


## Question 6

Load same data into H2O-3 dataframe.  
Describe imported dataframe.

In [13]:
import h2o
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,21 hours 47 mins
H2O_cluster_timezone:,America/Chicago
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.0.1
H2O_cluster_version_age:,1 month and 8 days
H2O_cluster_name:,H2O_from_python_rahulmullaguru_buv9u7
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,634 Mb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [14]:
df_h = h2o.import_file('./data/SBA_loans_small.zip')

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [15]:
df_h.describe()

Rows:448583
Cols:20




,City,State,Zip,Bank,BankState,NAICS,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementGross,BalanceGross,MIS_Status,GrAppv,SBA_Appv
type,enum,enum,int,enum,enum,int,int,int,int,int,int,int,int,enum,enum,enum,int,enum,enum,enum
mins,,,0.0,,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,,,,0.0,,,
mean,,,53841.996368565015,,,398694.6823620155,110.78555807955273,11.406569129904613,1.2799479166666652,8.606342638931926,10.968233303535797,2760.593203487427,0.7566314372145179,,,,0.0,,,
maxs,,,99999.0,,,928120.0,527.0,9999.0,2.0,8800.0,8800.0,99999.0,2.0,,,,0.0,,,
sigma,,,31169.478015031855,,,263264.92563379975,78.85420615377278,72.25402961839185,0.4515142294208939,239.71012479114094,240.13097630419097,12783.228481275728,0.6458408354967028,,,,0.0,,,
zeros,,,139,,,100755,394,3318,513,313701,219984,103706,161424,,,,448581,,,
missing,13,6,0,755,757,0,0,0,71,0,0,0,0,130648,2037,0,2,0,0,0
0,BUFFALO,NY,14218.0,BANK OF AMERICA NATL ASSOC,NC,812111.0,2.0,7.0,1.0,0.0,0.0,1.0,1.0,Y,N,"$74,104.00",0.0,CHGOFF,"$16,000.00","$8,000.00"
1,WEST MIFFLIN,PA,15122.0,CITIZENS BANK NATL ASSOC,RI,236115.0,2.0,1.0,1.0,0.0,1.0,1.0,1.0,Y,N,"$76,955.00",0.0,CHGOFF,"$35,000.00","$17,500.00"
2,SAN DIEGO,CA,92126.0,MUFG UNION BANK NATL ASSOC,CA,442110.0,5.0,6.0,1.0,0.0,6.0,1.0,1.0,,N,"$24,985.00",0.0,CHGOFF,"$25,000.00","$12,500.00"


In [16]:
df_h.head()

City,State,Zip,Bank,BankState,NAICS,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementGross,BalanceGross,MIS_Status,GrAppv,SBA_Appv
BUFFALO,NY,14218,BANK OF AMERICA NATL ASSOC,NC,812111,2,7,1,0,0,1,1,Y,N,"$74,104.00",0,CHGOFF,"$16,000.00","$8,000.00"
WEST MIFFLIN,PA,15122,CITIZENS BANK NATL ASSOC,RI,236115,2,1,1,0,1,1,1,Y,N,"$76,955.00",0,CHGOFF,"$35,000.00","$17,500.00"
SAN DIEGO,CA,92126,MUFG UNION BANK NATL ASSOC,CA,442110,5,6,1,0,6,1,1,,N,"$24,985.00",0,CHGOFF,"$25,000.00","$12,500.00"
EDMONDS,WA,98026,BANK OF AMERICA NATL ASSOC,NC,333912,32,1,1,0,1,0,1,Y,N,"$7,265.00",0,CHGOFF,"$5,000.00","$2,500.00"
PHILADELPHIA,PA,19120,"PNC BANK, NATIONAL ASSOCIATION",DE,812320,58,1,1,1,1,1,1,Y,N,"$9,972.00",0,CHGOFF,"$10,000.00","$5,000.00"
SAINT LOUIS,MO,63139,U.S. BANK NATIONAL ASSOCIATION,OH,442299,29,4,1,0,4,0,1,Y,N,"$53,654.00",0,CHGOFF,"$20,000.00","$10,000.00"
LAFAYETTE,LA,70507,HOME BANK,LA,621320,82,5,2,0,0,1,1,N,Y,"$91,500.00",0,CHGOFF,"$91,500.00","$77,775.00"
VALENCIA,CA,91402,BANK OF AMERICA NATL ASSOC,NC,337110,49,3,1,10,3,1,1,T,N,"$20,000.00",0,CHGOFF,"$20,000.00","$10,000.00"
BROOKLYN,NY,11237,JPMORGAN CHASE BANK NATL ASSOC,IL,561510,29,6,1,1,6,0,1,N,N,"$40,400.00",0,CHGOFF,"$40,400.00","$20,200.00"
Richmond,VA,23238,SUNTRUST BANK,GA,812199,32,4,2,4,0,0,1,Y,N,"$47,499.00",0,CHGOFF,"$25,000.00","$12,500.00"


## Question 7

Use `fillna` method to fill missing values for the following columns:
- City
- State
- Bank
- BankState
- NewExist
- RevLineCr
- LowDoc

https://docs.h2o.ai/h2o/latest-stable/h2o-docs/data-munging/fillnas.html

In [17]:
'''All Dataframe columns '''
df_h.columns

NameError: name 'df_h' is not defined

In [18]:
df_h = df_h.fillna(method="forward")

NameError: name 'df_h' is not defined

In [19]:
df_h.head()

NameError: name 'df_h' is not defined

## Question 8

Perform target encoding on the following columns:
- ZIP
- City
- State

https://docs.h2o.ai/h2o/latest-stable/h2o-docs/data-science/target-encoding.html


In [20]:
#df_h["Zip_encoded"] = df_h.groupby("Zip")["MIS_Status"].transform("mean")
#df_h["City_encoded"] = df_h.groupby("City")["MIS_Status"].transform("mean")
#df_h["State_encoded"] = df_h.groupby("State")["MIS_Status"].transform("mean")

In [21]:
#pip install category_encoders

In [22]:
from h2o.estimators import H2OTargetEncoderEstimator
df_h['MIS_Status'] = df_h['MIS_Status'].asfactor()
response='MIS_Status'
train, test = df_h.split_frame(ratios = [.8], seed = 1234)
df_h['Zip'] = df_h['Zip'].asfactor()
encoded_columns = ['Zip','City', 'State']
fold_column = "kfold_column"
train[fold_column] = train.kfold_column(n_folds=5, seed=5678)

# Train a Target Encoded model
df_h_te = H2OTargetEncoderEstimator(fold_column=fold_column,
                                       data_leakage_handling="k_fold",
                                       blending=True,
                                       inflection_point=3,
                                       smoothing=8,
                                       noise=0.15)   

df_h_te.train(x=encoded_columns,
                 y=response,
                 training_frame=train)

targetencoder Model Build progress: |████████████████████████████████████████████| (done) 100%
Model Details
H2OTargetEncoderEstimator :  TargetEncoder
Model Key:  TargetEncoder_model_python_1644133603530_38


Target Encoder model summary: Summary for target encoder model


,original_names,encoded_column_names
0,City,City_te
1,State,State_te
2,Zip,Zip_te
